In [1]:
import devito as dv

In [2]:
grid = dv.Grid(shape=(201, 201), extent=(1., 1.))
p = dv.TimeFunction(name='p', grid=grid, space_order=2)
v = dv.VectorTimeFunction(name='v', grid=grid, space_order=2, staggered=(None, None))

# Material parameters
c = dv.Function(name='c', grid=grid)
rho = dv.Function(name='rho', grid=grid)
c.data[:] = 1.
rho.data[:] = 1.

# Define bouyancy for shorthand
b = 1/rho
# Define square celerity for shorthand
c2 = c**2  # FIXME: This prints missing the ^2 (string does contain **2 however)

In [3]:
dv.grad(1/p)

Vector(Derivative(1/p(t, x, y), x), Derivative(1/p(t, x, y), y))

In [4]:
dt = grid.stepping_dim.spacing
dt

dt

In [5]:
# For setting up the 2nd-order ADER scheme

# First time derivatives
pdt = rho*c2*dv.div(v)
vdt = b*dv.grad(p)

# Second time derivatives
pdt2 = rho*c2*(dv.grad(b).dot(dv.grad(p)) + b*p.laplace)
vdt2 = b*(dv.grad(rho)*c2*dv.div(v) + rho*dv.grad(c2)*dv.div(v) + rho*c2*dv.grad(dv.div(v)))
vdt2  # FIXME: dv.grad(dv.div(v)) does not get flattened and will need to be explicitly written out

Vector(((Derivative(v_x(t, x, y), x) + Derivative(v_y(t, x, y), y))*c(x, y)**2*Derivative(rho(x, y), x) + (Derivative(v_x(t, x, y), x) + Derivative(v_y(t, x, y), y))*rho(x, y)*Derivative(c(x, y)**2, x) + c(x, y)**2*rho(x, y)*Derivative(Derivative(v_x(t, x, y), x) + Derivative(v_y(t, x, y), y), x))/rho(x, y), ((Derivative(v_x(t, x, y), x) + Derivative(v_y(t, x, y), y))*c(x, y)**2*Derivative(rho(x, y), y) + (Derivative(v_x(t, x, y), x) + Derivative(v_y(t, x, y), y))*rho(x, y)*Derivative(c(x, y)**2, y) + c(x, y)**2*rho(x, y)*Derivative(Derivative(v_x(t, x, y), x) + Derivative(v_y(t, x, y), y), y))/rho(x, y))

In [6]:
# Update equations (2nd-order ADER timestepping)
eq_p = dv.Eq(p.forward, p + dt*pdt + (dt**2/2)*pdt2)
eq_v = dv.Eq(v.forward, v + dt*vdt + (dt**2/2)*vdt2)